In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/25 23:10:36 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.27.116 instead (on interface eth0)
22/08/25 23:10:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 23:10:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/25 23:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_indexed')
yellow_sample = pd.read_parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')

In [4]:
yellow_sample

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,tolls_amount,total_amount,is_weekend,...,Date,Start(ET),Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,is_weekend_binary,Win_binary,Start(ET)_NUMERIC
0,2018-12-25 10:29:03,2018-12-25 10:31:40,0.63,100,48,4.0,0.96,0.0,5.76,False,...,2018-12-25,12:00,19812,No,-14,10,10,0,0,1.0
1,2018-12-25 19:48:14,2018-12-25 19:51:23,0.71,114,234,4.5,1.06,0.0,6.36,False,...,2018-12-25,12:00,19812,No,-14,19,19,0,0,1.0
2,2018-12-25 00:03:23,2018-12-25 00:17:02,2.30,234,144,11.0,1.48,0.0,13.78,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
3,2018-12-25 00:07:38,2018-12-25 00:18:00,2.20,239,75,9.5,1.75,0.0,12.55,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
4,2018-12-25 00:00:30,2018-12-25 00:10:02,1.68,238,143,8.5,2.94,0.0,12.74,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616159,2019-03-24 23:47:22,2019-03-24 23:55:36,1.68,186,224,8.0,2.36,0.0,14.16,True,...,2019-03-24,12:00,18263,No,-11,23,23,1,0,1.0
616160,2019-03-24 23:54:14,2019-03-24 23:58:38,1.38,41,152,6.0,1.00,0.0,8.30,True,...,2019-03-24,12:00,18263,No,-11,23,23,1,0,1.0
616161,2019-03-24 23:41:04,2019-03-24 23:44:41,0.80,230,100,4.5,1.65,0.0,9.95,True,...,2019-03-24,12:00,18263,No,-11,23,23,1,0,1.0
616162,2019-03-24 23:54:40,2019-03-24 23:58:35,1.02,144,113,5.5,1.86,0.0,11.16,True,...,2019-03-24,12:00,18263,No,-11,23,23,1,0,1.0


In [5]:
# standardized the x columns code from tutorial 2


from scipy.stats import zscore

x_cols = ['fare_amount','tolls_amount', 'total_amount', 'is_weekend_binary', 'trip_length', 
          'Attendance', 'Win_binary', 'margin_victory/loss','pickup_hour', 'dropoff_hour', 
          'PULocationID', 'DOLocationID', 'Start(ET)_NUMERIC']

y_cols = ['tip_amount']

df_standard = yellow_sample[x_cols].astype(float).apply(zscore)

# format output to 4 decimal places
pd.options.display.float_format = '{:,.4f}'.format
df_standard.describe().loc[['mean','std']]



,fare_amount,tolls_amount,total_amount,is_weekend_binary,trip_length,Attendance,Win_binary,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID,DOLocationID,Start(ET)_NUMERIC
mean,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [6]:
# fit the lasso model code from tutorial 2

import numpy as np
from glmnet import ElasticNet

elastic_net_model_lasso = ElasticNet(alpha=1) 
elastic_net_model_lasso.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    yellow_sample[y_cols].values.flatten()
)

ElasticNet()

In [7]:
# fit the ridge model code from tutorial 2

elastic_net_model_ridge = ElasticNet(alpha=0) 
elastic_net_model_ridge.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    yellow_sample[y_cols].values.flatten()
)

ElasticNet(alpha=0)

In [8]:
best_lambda_lasso = elastic_net_model_lasso.lambda_best_[0]
best_lambda_ridge = elastic_net_model_ridge.lambda_best_[0]

print(f'Best lambda value for LASSO:', best_lambda_lasso)
print(f'Best lambda value for Ridge:', best_lambda_ridge)


Best lambda value for LASSO: 0.0035325405143202923
Best lambda value for Ridge: 0.10297543045823393


In [9]:
# drop the tpep pickup and dropoff columns, and categorical columns which have not been indexed

yellow = yellow.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Start(ET)',
                     'Win', 'is_weekend')

In [10]:
# use all 2018 data as a train set

# then use the first game of 2019 as the test set

yellow_train = yellow.where((F.col('Date') < '2019-01-11'))

yellow_test = yellow.where((F.col('Date') == '2019-01-11'))


In [11]:
# drop Date columns for both train and test

yellow_train = yellow_train.drop('Date')

yellow_test = yellow_test.drop('Date')

In [12]:
# create a vector for the train dataframe ridge model

from pyspark.ml.feature import VectorAssembler

features = 'features'

# use all featuresn
input_cols_full = ['fare_amount','tolls_amount', 'trip_distance', 'total_amount', 'is_weekend_binary', 'trip_length', 
                   'Win_binary','PULocationID', 'DOLocationID', 'pickup_hour', 'dropoff_hour','Attendance', 
                    'margin_victory/loss', 'Start(ET)_NUMERIC']

vectorAssembler_full = VectorAssembler(
    inputCols = input_cols_full, 
    outputCol = features)


v_yellow_train_full = vectorAssembler_full.transform(yellow_train.dropna('any'))

v_yellow_train_full = v_yellow_train_full.select(['features', 'tip_amount'])

v_yellow_train_full.show(3)

22/08/25 23:11:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,0.0,2.56,22...|      4.58|
|[5.0,0.0,0.9,7.25...|      1.45|
|[4.0,0.0,0.63,5.7...|      0.96|
+--------------------+----------+
only showing top 3 rows



In [13]:
v_yellow_test_full = vectorAssembler_full.transform(yellow_test.dropna('any'))

v_yellow_test_full = v_yellow_test_full.select(['features', 'tip_amount'])

v_yellow_test_full.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,0.0,2.88,18...|       3.7|
|(14,[0,2,3,5,7,8,...|      2.26|
|(14,[0,2,3,5,7,8,...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [14]:
# LassoM stands for LASSO model

from pyspark.ml.regression import LinearRegression

LassoM = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_lasso, elasticNetParam = 1).fit(v_yellow_train_full)

22/08/25 23:11:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/25 23:11:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [15]:
#coefficients for LASSO regression

pd.DataFrame(
    data=[LassoM.intercept] + list(LassoM.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

# based on LASSO, Attendance, margin_victory/loss, Win_binary, and trip_length are all irrelevant

,coefficient
intercept,-0.7906
fare_amount,-0.8235
tolls_amount,-0.8216
trip_distance,-0.0365
total_amount,0.8572
is_weekend_binary,0.1266
trip_length,0.0000
Win_binary,0.0000
PULocationID,0.0001
DOLocationID,0.0001


In [18]:
features = 'features'

# features selected after selection through LASSO
input_cols_reduced = ['fare_amount', 'trip_distance', 'total_amount', 'tolls_amount', 'is_weekend_binary',
                     'PULocationID', 'DOLocationID', 'pickup_hour','dropoff_hour', 'Start(ET)_NUMERIC']


vectorAssembler_reduced = VectorAssembler(
    inputCols = input_cols_reduced, 
    outputCol = features)


v_yellow_train_reduced = vectorAssembler_reduced.transform(yellow_train.dropna('any'))

v_yellow_train_reduced = v_yellow_train_reduced.select(['features', 'tip_amount'])

v_yellow_train_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[5.0,0.9,7.25,0.0...|      1.45|
|[4.0,0.63,5.76,0....|      0.96|
+--------------------+----------+
only showing top 3 rows



In [19]:
v_yellow_test_reduced = vectorAssembler_reduced.transform(yellow_test.dropna('any'))

v_yellow_test_reduced = v_yellow_test_reduced.select(['features', 'tip_amount'])

v_yellow_test_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|(10,[0,1,2,5,6],[...|      2.26|
|(10,[0,1,2,5,6],[...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [20]:
lm = LinearRegression(featuresCol='features', labelCol = 'tip_amount').fit(v_yellow_train_reduced)

22/08/25 23:11:26 WARN Instrumentation: [97d1afe2] regParam is zero, which might cause numerical instability and overfitting.


22/08/25 23:11:28 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [21]:
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols_reduced,
    columns=['coefficient']
)

,coefficient
intercept,-0.8346
fare_amount,-0.8597
trip_distance,-0.0392
total_amount,0.8881
tolls_amount,-0.8666
is_weekend_binary,0.1391
PULocationID,0.0001
DOLocationID,0.0002
pickup_hour,-0.0239
dropoff_hour,0.0066


In [22]:
trainingSummary_lm = lm.summary

print("RMSE: %f" % trainingSummary_lm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_lm.r2)
print("MSE: %f" % trainingSummary_lm.meanSquaredError)
print("MAE: %f" % trainingSummary_lm.meanAbsoluteError)


RMSE: 0.330728
r2: 0.934314
MSE: 0.109381
MAE: 0.266606


In [23]:
from pyspark.ml.evaluation import RegressionEvaluator


lm_predictions = lm.transform(v_yellow_test_reduced)
lm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|4.2343350961999136|       3.7|[13.0,2.88,18.5,0...|
| 2.559191494822122|      2.26|(10,[0,1,2,5,6],[...|
| 2.182411548245235|      1.86|(10,[0,1,2,5,6],[...|
| 2.713104899806301|      2.49|(10,[0,1,2,5,6],[...|
| 4.304380112562934|      4.06|(10,[0,1,2,5,6],[...|
+------------------+----------+--------------------+
only showing top 5 rows



In [24]:
test_result_lm = lm.evaluate(v_yellow_test_reduced)

print("R Squared (R2) on test data = %g" % test_result_lm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_lm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_lm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_lm.meanAbsoluteError)

R Squared (R2) on test data = 0.931308
Root Mean Squared Error (RMSE) on test data = 0.352211
Mean Squared Error (MSE) on test data = 0.124052
Mean Absolute Error (MAE) on test data = 0.286736


In [25]:
# rm stands for ridge model

rm = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_ridge, elasticNetParam = 0).fit(v_yellow_train_full)

In [26]:
pd.DataFrame(
    data=[rm.intercept] + list(rm.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

,coefficient
intercept,-0.1003
fare_amount,-0.1062
tolls_amount,-0.1181
trip_distance,-0.0643
total_amount,0.2628
is_weekend_binary,0.0033
trip_length,-0.0062
Win_binary,0.0002
PULocationID,0.0001
DOLocationID,0.0001


In [27]:
trainingSummary_rm = rm.summary

print("RMSE: %f" % trainingSummary_rm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_rm.r2)
print("MSE: %f" % trainingSummary_rm.meanSquaredError)
print("MAE: %f" % trainingSummary_rm.meanAbsoluteError)


RMSE: 0.779622
r2: 0.634995
MSE: 0.607811
MAE: 0.396092


In [28]:
rm_predictions = rm.transform(v_yellow_test_full)
rm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|3.0478409835051314|       3.7|[13.0,0.0,2.88,18...|
|2.1390481474108194|      2.26|(14,[0,2,3,5,7,8,...|
|1.7801290950764201|      1.86|(14,[0,2,3,5,7,8,...|
| 1.809341217883335|      2.49|(14,[0,2,3,5,7,8,...|
|3.7955189477644025|      4.06|(14,[0,2,3,5,7,8,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [29]:
test_result_rm = rm.evaluate(v_yellow_test_full)

print("R Squared (R2) on test data = %g" % test_result_rm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_rm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_rm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_rm.meanAbsoluteError)

R Squared (R2) on test data = 0.608752
Root Mean Squared Error (RMSE) on test data = 0.840572
Mean Squared Error (MSE) on test data = 0.706561
Mean Absolute Error (MAE) on test data = 0.36759
